# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "dadosabertosreceita"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-15 07:35:32,508 - INFO - Iniciando Sessão Spark.


# Parâmetros para leitura do arquivo

In [3]:
schema = T.StructType([
    T.StructField("codigo", T.StringType(), True),
    T.StructField("descricao", T.StringType(), True),
])

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("dadosabertosreceita")
local = f"s3a://landing/dadosabertosreceita/{ano_mes}/municipios/*.csv"

In [5]:
config_adicional = {
    'sep': ';',
    'encoding': 'latin1'
}

# Leitura do arquivo

In [6]:
df_compliance_receita_municipios = extrator_bronze.criar_view_temporaria_arquivo(local,'compliance_municipios', schema, **config_adicional)

2024-10-15 07:35:36,616 - INFO - Lista de yaml carregada com sucesso
2024-10-15 07:35:40,028 - INFO - Criando view temporaria.


# Gravação no datalake

In [7]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/dadosabertosreceita/municios'

In [8]:
extrator_bronze.salvar_delta(df_compliance_receita_municipios, 'overwrite')

2024-10-15 07:35:40,039 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-15 07:35:52,975 - INFO - Dados persistidos com sucesso
2024-10-15 07:35:52,977 - INFO - s3a://bronze/dadosabertosreceita/municios
2024-10-15 07:35:52,977 - INFO - Aguarde... Realizando optimize
2024-10-15 07:36:01,046 - INFO - Optimize executado com sucesso.
2024-10-15 07:36:01,047 - INFO - Aguarde... Realizando vacuum
2024-10-15 07:36:26,442 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [9]:
extrator_bronze.parar_sessao()

2024-10-15 07:36:27,049 - INFO - Sessão Spark finalizada.
